In [55]:
# one needs to import those packages which are needed; best to be done at the beginning of the program.
import networkx as nx

import numpy as np
import scipy as sp
from scipy.linalg import expm, sinm, cosm
import pandas as pd
# some basic settings for plotting figures
import matplotlib.pyplot as plt
%matplotlib inline 
font = {'family' : 'DejaVu Sans',
        'weight' : 'bold',
        'size'   : 32}

plt.rc('font', **font)

In [81]:
def subgraph(A,i,j):
    "This returns the i, j component of the subgraph centrality matrix"
    from numpy import linalg as LA
    w, v = LA.eig(A)
    w=[l/max(w,key=abs) for l in w]
    expdiag=[np.exp(l) for l in w]
    intermediate=np.matmul(v,np.diag(expdiag))
    subgraphmat=np.matmul(intermediate,np.linalg.inv(v))

    print(subgraphmat[i,j])
    return;
   

In [57]:

# on your laptop it might be:
G0 = nx.read_weighted_edgelist("C:\\Users\\cajic\\Downloads\\4932.protein.links.v11.5.txt",comments="#",nodetype=str) #Delete the header first or this might not work!


In [58]:
# delete those edges with a combined score of <= threshold_score (small confidence)
threshold_score = 750
for edge in G0.edges: 
    weight = list(G0.get_edge_data(edge[0],edge[1]).values())
    if(weight[0] <= threshold_score):
        G0.remove_edge(edge[0],edge[1])

In [6]:
# compute degree sequence
degS=[G0.degree()[node] for node in list(G0.nodes())]
degS.sort()
degS=np.array(degS)

In [7]:
#Number of nodes with degree 1
count=0
for x in degS:
    if x==1:
        count=count+1 
count

284

In [8]:
#4 nodes with largest degree
indices=np.argpartition(degS, -4)[-4:]
indices

array([5717, 5718, 5719, 5720], dtype=int64)

In [59]:
#remove essential nodes from graph G, then get largest connected component

G0nodes=list(G0.nodes())
esnodelist=[]
nodes_to_remove=[]

esnodes=pd.read_table('C:\\Users\\cajic\\Downloads\\essential_nodes.csv',sep=',')

for node in esnodes['B']:
    esnodelist.append('4932.'+node)
    
G0=nx.Graph(G0) #unfreeze graph

for x in esnodelist:
    if G0nodes.count(x)>0:
        G0.remove_node(x)
        
#get the largest component
largest_cc = max(nx.connected_components(G0),key=len)
G = G0.subgraph(largest_cc)
Adj = nx.to_numpy_array(G)
print('number of nodes of largest connected subgraph of G0:',G.number_of_nodes())
print('number of edges of largest connected subgraph of G0:',G.number_of_edges())
len(G.nodes())

number of nodes of largest connected subgraph of G0: 4396
number of edges of largest connected subgraph of G0: 44311


4396

In [60]:
#assign communities using louvain (already assigned, do not run louvain again because it will give a different result)
import community as community_louvain
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import networkx as nx

# load the karate club graph
#G = nx.karate_club_graph()

# first compute the best partition

#sdnorm=[]
#for i in range(0,20):
#    partnums=[]
#    for j in range(50):
#        partition = community_louvain.best_partition(G,resolution=1+i/20)
#        partnums.append(max(list(partition.values()))+1)
#    sd=np.std(partnums)
#    av=np.mean(partnums)
#    sdnorm.append(sd/av)
#minsd=min(sdnorm)
#minidx=sdnorm.index(minsd)
#print('optimum resolution is', 1+minidx/20)
#unique = list(dict.fromkeys(list(partition.values())))

#optimum resolution not consistent, but around 1, so take 1.  
#partition = community_louvain.best_partition(G,resolution=1)
# draw the graph
#pos = nx.spring_layout(G)
# color the nodes according to their partition
#cmap = cm.get_cmap('viridis', max(partition.values()) + 1)
#nx.draw_networkx_nodes(G, pos, partition.keys(), node_size=40,
                       #cmap=cmap, node_color=list(partition.values()))
#nx.draw_networkx_edges(G, pos, alpha=0.5)
#plt.show()
partvals=[0, 1, 0, 2, 1, 3, 4, 0, 5, 6, 6, 0, 7, 0, 0, 5, 8, 9, 1, 0, 8, 0, 1, 6, 6, 5, 4, 4, 5, 4, 10, 9, 8, 3, 6, 11, 11, 6, 6, 6, 6, 6, 11, 6, 6, 6, 6, 6, 6, 6, 6, 11, 4, 5, 5, 11, 11, 6, 6, 6, 6, 6, 12, 4, 9, 6, 6, 9, 4, 6, 6, 6, 6, 12, 6, 4, 5, 6, 6, 6, 5, 6, 6, 4, 13, 14, 4, 3, 6, 6, 19, 4, 6, 6, 9, 6, 6, 6, 11, 4, 11, 6, 6, 7, 6, 6, 12, 3, 2, 6, 6, 4, 11, 5, 6, 6, 11, 6, 11, 6, 5, 6, 8, 4, 6, 4, 6, 13, 7, 6, 6, 5, 1, 11, 6, 11, 6, 6, 5, 4, 6, 6, 4, 4, 6, 4, 7, 6, 6, 6, 6, 16, 6, 6, 4, 4, 6, 6, 6, 13, 6, 11, 2, 6, 6, 6, 6, 4, 6, 4, 12, 1, 2, 6, 11, 6, 6, 6, 6, 6, 4, 4, 6, 4, 1, 4, 4, 4, 3, 11, 6, 6, 7, 6, 12, 3, 6, 19, 10, 6, 2, 11, 6, 6, 11, 6, 6, 4, 6, 11, 9, 4, 6, 6, 6, 11, 6, 4, 11, 11, 6, 4, 6, 4, 6, 9, 1, 1, 6, 6, 11, 6, 7, 6, 4, 6, 11, 6, 6, 11, 5, 6, 17, 11, 6, 6, 6, 9, 6, 10, 11, 4, 6, 6, 6, 3, 11, 6, 6, 5, 11, 19, 6, 1, 5, 7, 6, 4, 11, 4, 6, 6, 11, 3, 11, 6, 17, 6, 4, 11, 6, 3, 6, 6, 6, 17, 6, 6, 4, 6, 11, 6, 10, 6, 6, 4, 4, 6, 9, 4, 6, 7, 7, 7, 18, 4, 7, 5, 4, 4, 4, 7, 4, 5, 7, 4, 2, 4, 7, 9, 8, 17, 4, 1, 7, 5, 12, 7, 11, 6, 9, 7, 4, 6, 6, 11, 6, 6, 6, 7, 6, 6, 16, 11, 2, 6, 18, 6, 12, 6, 6, 11, 12, 6, 11, 11, 5, 16, 6, 6, 5, 16, 6, 6, 18, 14, 6, 6, 6, 16, 10, 6, 6, 3, 18, 11, 6, 6, 6, 16, 6, 9, 2, 12, 6, 18, 10, 11, 14, 7, 10, 2, 5, 16, 6, 4, 5, 4, 5, 11, 5, 4, 8, 8, 4, 5, 5, 4, 5, 4, 11, 5, 4, 5, 5, 9, 11, 17, 7, 9, 11, 14, 9, 5, 12, 4, 4, 6, 11, 6, 11, 5, 12, 4, 12, 4, 5, 17, 4, 4, 11, 5, 11, 5, 6, 4, 17, 9, 5, 5, 2, 6, 6, 5, 7, 10, 5, 11, 8, 4, 5, 4, 10, 12, 14, 7, 6, 10, 2, 7, 6, 14, 4, 5, 5, 5, 5, 10, 5, 11, 5, 11, 11, 4, 14, 11, 4, 14, 11, 11, 4, 11, 4, 17, 4, 9, 12, 16, 11, 11, 5, 6, 12, 5, 11, 16, 5, 4, 4, 11, 11, 9, 6, 10, 2, 10, 11, 6, 10, 9, 5, 5, 2, 11, 4, 4, 12, 11, 5, 5, 4, 4, 4, 1, 4, 5, 4, 4, 9, 4, 14, 6, 14, 4, 11, 8, 9, 10, 11, 5, 5, 4, 5, 6, 7, 4, 11, 11, 5, 11, 5, 6, 9, 1, 11, 7, 8, 12, 7, 5, 4, 5, 10, 17, 9, 4, 8, 2, 11, 4, 5, 11, 11, 4, 4, 2, 6, 6, 16, 5, 4, 2, 4, 6, 7, 18, 7, 7, 7, 7, 7, 7, 10, 7, 7, 7, 9, 9, 6, 16, 6, 9, 6, 9, 4, 5, 12, 6, 10, 4, 18, 7, 4, 6, 6, 6, 6, 3, 6, 12, 4, 6, 6, 6, 5, 11, 4, 6, 16, 6, 6, 4, 7, 1, 12, 6, 6, 4, 9, 2, 4, 5, 5, 18, 4, 18, 11, 4, 18, 9, 6, 5, 1, 3, 2, 9, 6, 2, 11, 11, 5, 8, 17, 5, 3, 5, 16, 5, 2, 4, 4, 5, 2, 4, 5, 16, 2, 16, 5, 2, 4, 9, 6, 5, 16, 8, 6, 5, 2, 16, 11, 1, 6, 5, 6, 5, 3, 11, 2, 19, 5, 5, 7, 10, 7, 5, 1, 14, 4, 19, 16, 16, 2, 4, 14, 3, 10, 6, 5, 5, 4, 20, 12, 12, 5, 5, 5, 5, 5, 11, 2, 5, 3, 12, 5, 6, 8, 6, 2, 16, 1, 11, 16, 12, 2, 5, 2, 2, 5, 5, 5, 6, 6, 2, 2, 17, 4, 9, 6, 5, 6, 2, 4, 5, 6, 6, 1, 4, 4, 14, 6, 2, 5, 5, 3, 5, 2, 2, 16, 5, 2, 6, 16, 5, 16, 5, 5, 5, 16, 11, 7, 2, 11, 2, 11, 11, 11, 2, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 5, 11, 11, 11, 11, 11, 11, 4, 11, 11, 11, 11, 11, 11, 11, 12, 1, 3, 7, 2, 5, 12, 10, 1, 12, 1, 7, 7, 7, 18, 10, 18, 4, 7, 7, 7, 7, 7, 4, 7, 3, 16, 7, 7, 4, 11, 12, 7, 7, 3, 7, 4, 7, 7, 1, 7, 8, 4, 4, 10, 6, 2, 4, 8, 7, 4, 7, 7, 7, 7, 7, 8, 7, 10, 4, 7, 8, 3, 12, 4, 10, 18, 1, 4, 4, 7, 7, 7, 4, 7, 4, 7, 7, 10, 7, 7, 20, 4, 10, 8, 7, 12, 4, 8, 3, 10, 7, 7, 7, 3, 10, 7, 7, 12, 12, 7, 1, 11, 7, 4, 4, 12, 1, 4, 2, 6, 16, 11, 8, 2, 10, 4, 8, 12, 3, 12, 7, 2, 6, 11, 12, 9, 10, 3, 16, 6, 12, 4, 8, 12, 1, 6, 12, 12, 4, 8, 4, 12, 12, 8, 9, 19, 6, 4, 10, 4, 9, 6, 8, 1, 16, 1, 12, 1, 12, 4, 6, 8, 1, 12, 12, 5, 16, 4, 7, 4, 9, 10, 6, 8, 11, 8, 6, 9, 4, 12, 12, 4, 12, 4, 14, 4, 10, 5, 18, 16, 4, 4, 12, 7, 12, 5, 5, 4, 4, 14, 8, 7, 2, 12, 2, 6, 6, 10, 12, 4, 12, 4, 4, 8, 3, 12, 3, 8, 8, 9, 9, 16, 3, 4, 3, 9, 3, 11, 3, 17, 2, 3, 10, 5, 2, 5, 5, 18, 2, 5, 4, 15, 9, 1, 16, 8, 4, 20, 4, 1, 9, 10, 1, 12, 11, 10, 4, 5, 7, 4, 12, 4, 4, 12, 6, 12, 12, 17, 16, 16, 3, 9, 14, 4, 9, 3, 7, 3, 12, 6, 10, 16, 9, 8, 16, 14, 11, 9, 10, 3, 10, 9, 6, 16, 10, 10, 10, 12, 16, 12, 16, 5, 4, 4, 18, 12, 16, 10, 5, 4, 10, 10, 8, 12, 12, 12, 9, 18, 12, 16, 3, 16, 3, 16, 12, 16, 3, 8, 7, 16, 16, 16, 16, 7, 1, 3, 19, 16, 12, 16, 16, 16, 16, 10, 3, 16, 7, 8, 16, 3, 14, 9, 2, 8, 16, 8, 19, 12, 1, 16, 9, 16, 16, 16, 16, 8, 16, 16, 12, 8, 12, 3, 16, 19, 16, 16, 8, 12, 16, 16, 16, 10, 16, 12, 16, 16, 12, 16, 14, 7, 15, 3, 16, 16, 3, 16, 3, 16, 12, 16, 16, 16, 16, 10, 16, 14, 12, 3, 3, 16, 19, 16, 16, 8, 16, 3, 16, 16, 16, 12, 16, 16, 12, 19, 16, 12, 12, 7, 3, 16, 16, 8, 16, 16, 7, 10, 10, 16, 16, 8, 10, 16, 2, 16, 16, 16, 16, 16, 5, 8, 16, 16, 16, 4, 16, 19, 16, 16, 12, 16, 16, 1, 16, 16, 12, 16, 18, 16, 16, 16, 4, 3, 16, 16, 3, 9, 16, 3, 6, 16, 3, 16, 16, 16, 16, 8, 16, 16, 10, 3, 16, 3, 7, 16, 16, 16, 3, 16, 8, 9, 12, 16, 16, 6, 10, 16, 12, 8, 16, 6, 12, 16, 12, 9, 16, 3, 8, 16, 16, 8, 16, 16, 16, 3, 16, 10, 16, 12, 3, 16, 12, 16, 12, 16, 16, 12, 8, 5, 12, 5, 17, 7, 3, 5, 5, 10, 6, 5, 1, 10, 7, 4, 10, 10, 5, 9, 4, 10, 5, 8, 4, 10, 17, 9, 12, 5, 5, 9, 4, 12, 16, 7, 9, 17, 12, 10, 12, 4, 17, 8, 17, 5, 10, 17, 6, 5, 10, 10, 5, 4, 10, 8, 5, 5, 5, 4, 8, 11, 10, 5, 5, 12, 17, 14, 10, 5, 9, 8, 4, 17, 4, 10, 12, 12, 10, 4, 4, 5, 2, 10, 5, 10, 10, 12, 10, 10, 10, 10, 11, 4, 12, 10, 5, 10, 17, 10, 4, 7, 5, 11, 5, 3, 17, 10, 9, 5, 8, 4, 5, 5, 10, 12, 10, 10, 12, 4, 4, 7, 10, 12, 5, 4, 12, 5, 7, 5, 10, 4, 4, 12, 5, 5, 9, 17, 5, 5, 5, 12, 3, 19, 12, 4, 5, 16, 4, 5, 5, 12, 3, 7, 7, 5, 4, 4, 11, 8, 12, 8, 3, 4, 12, 16, 7, 10, 12, 10, 10, 2, 4, 12, 10, 4, 12, 1, 12, 5, 5, 12, 9, 18, 12, 18, 16, 18, 12, 8, 18, 14, 12, 12, 12, 12, 16, 7, 16, 12, 4, 12, 18, 16, 12, 16, 8, 18, 6, 18, 12, 18, 8, 8, 4, 10, 12, 3, 11, 16, 16, 8, 7, 5, 9, 16, 17, 12, 2, 12, 4, 12, 16, 10, 16, 4, 8, 12, 9, 8, 3, 8, 5, 9, 12, 12, 10, 8, 10, 10, 4, 19, 10, 7, 10, 9, 7, 10, 10, 5, 8, 10, 3, 3, 12, 4, 3, 8, 20, 3, 10, 16, 16, 2, 12, 12, 4, 4, 10, 3, 4, 3, 2, 8, 8, 8, 12, 10, 8, 4, 7, 8, 12, 4, 10, 2, 8, 10, 8, 17, 5, 10, 10, 9, 3, 8, 16, 9, 12, 12, 12, 3, 5, 6, 9, 12, 10, 12, 2, 8, 8, 8, 9, 7, 9, 1, 9, 7, 10, 14, 10, 14, 10, 9, 12, 17, 4, 16, 7, 14, 7, 16, 8, 16, 9, 4, 9, 10, 10, 8, 12, 4, 8, 7, 3, 8, 12, 19, 9, 12, 10, 12, 8, 2, 7, 12, 1, 3, 9, 4, 2, 2, 12, 4, 8, 2, 12, 8, 7, 8, 4, 10, 8, 12, 19, 12, 8, 7, 4, 8, 12, 12, 8, 8, 8, 12, 3, 16, 4, 6, 5, 8, 9, 8, 14, 3, 9, 9, 4, 19, 10, 4, 3, 9, 12, 3, 8, 10, 3, 10, 1, 4, 9, 4, 5, 16, 16, 9, 17, 4, 7, 5, 19, 12, 5, 10, 10, 17, 7, 10, 10, 9, 10, 10, 9, 12, 9, 12, 2, 16, 10, 5, 4, 7, 7, 4, 4, 9, 16, 10, 5, 7, 9, 7, 12, 16, 3, 7, 3, 8, 4, 8, 12, 10, 1, 2, 12, 5, 4, 3, 2, 2, 6, 3, 12, 7, 4, 19, 7, 16, 12, 9, 5, 12, 12, 16, 12, 4, 4, 10, 8, 10, 10, 9, 12, 10, 4, 8, 3, 16, 10, 12, 8, 7, 4, 12, 12, 16, 20, 12, 12, 8, 16, 3, 8, 4, 10, 4, 4, 4, 5, 8, 19, 2, 7, 10, 9, 5, 12, 8, 16, 12, 16, 8, 1, 8, 16, 3, 12, 4, 10, 3, 18, 16, 2, 3, 14, 9, 10, 3, 12, 5, 17, 8, 12, 14, 8, 8, 12, 8, 16, 9, 19, 12, 8, 8, 8, 4, 8, 19, 16, 12, 8, 10, 12, 4, 8, 8, 3, 10, 16, 7, 10, 16, 16, 9, 16, 10, 12, 8, 12, 12, 8, 8, 8, 3, 8, 12, 8, 12, 8, 12, 5, 19, 8, 8, 8, 8, 12, 12, 19, 9, 9, 9, 2, 9, 6, 6, 9, 7, 19, 4, 9, 9, 9, 9, 9, 9, 2, 19, 4, 9, 9, 9, 9, 9, 7, 2, 16, 2, 12, 16, 12, 16, 9, 2, 2, 14, 2, 2, 12, 2, 10, 10, 12, 2, 2, 12, 8, 8, 16, 2, 3, 8, 2, 12, 12, 2, 12, 10, 2, 2, 16, 17, 12, 2, 10, 2, 2, 2, 7, 9, 2, 16, 12, 2, 2, 3, 6, 5, 2, 2, 19, 12, 2, 2, 2, 16, 10, 12, 3, 2, 8, 12, 17, 2, 2, 3, 10, 6, 2, 2, 7, 2, 9, 6, 12, 6, 8, 8, 6, 2, 6, 16, 6, 16, 6, 16, 6, 7, 6, 2, 16, 2, 4, 6, 12, 16, 6, 12, 2, 6, 16, 2, 2, 3, 3, 3, 7, 3, 5, 7, 3, 3, 8, 3, 7, 3, 3, 7, 6, 3, 3, 7, 7, 7, 3, 7, 3, 10, 10, 3, 7, 3, 1, 3, 7, 16, 9, 3, 7, 10, 3, 3, 3, 3, 17, 3, 3, 3, 3, 3, 10, 3, 3, 10, 10, 3, 16, 3, 10, 3, 7, 3, 3, 3, 3, 3, 3, 3, 3, 14, 3, 3, 7, 7, 3, 3, 3, 3, 3, 1, 4, 4, 4, 6, 4, 9, 4, 12, 17, 4, 4, 18, 10, 14, 1, 8, 4, 4, 11, 4, 4, 4, 14, 4, 4, 4, 2, 4, 4, 14, 17, 1, 4, 6, 4, 9, 4, 4, 8, 4, 4, 3, 4, 16, 4, 17, 9, 4, 18, 4, 2, 12, 4, 2, 4, 8, 1, 16, 3, 4, 4, 4, 4, 1, 4, 1, 4, 4, 3, 4, 4, 14, 9, 8, 4, 4, 18, 4, 1, 4, 4, 4, 12, 14, 4, 18, 7, 12, 4, 4, 12, 2, 14, 4, 4, 4, 4, 12, 4, 1, 1, 4, 12, 4, 1, 12, 19, 4, 4, 1, 4, 10, 4, 4, 4, 4, 3, 4, 10, 18, 9, 4, 2, 4, 4, 4, 9, 16, 3, 16, 14, 3, 8, 3, 3, 3, 8, 3, 3, 14, 6, 3, 17, 9, 3, 8, 3, 14, 8, 8, 12, 3, 14, 3, 16, 12, 16, 3, 3, 7, 6, 12, 8, 19, 8, 8, 7, 16, 3, 14, 20, 12, 4, 3, 8, 12, 17, 4, 17, 4, 17, 17, 3, 5, 20, 3, 3, 12, 8, 17, 10, 3, 4, 9, 12, 12, 12, 4, 12, 18, 6, 12, 18, 16, 12, 16, 12, 12, 16, 4, 9, 12, 12, 4, 12, 12, 7, 7, 5, 1, 10, 9, 5, 7, 3, 3, 8, 14, 10, 12, 12, 7, 16, 10, 4, 9, 9, 4, 7, 14, 6, 7, 8, 14, 7, 4, 4, 14, 3, 19, 1, 9, 12, 6, 2, 10, 19, 19, 8, 14, 4, 10, 4, 4, 4, 1, 7, 1, 12, 16, 16, 3, 7, 4, 9, 7, 4, 4, 10, 5, 5, 5, 7, 7, 1, 9, 5, 13, 10, 19, 12, 7, 7, 12, 7, 7, 2, 14, 7, 11, 10, 4, 12, 3, 7, 14, 7, 10, 12, 4, 10, 3, 14, 3, 20, 12, 14, 9, 12, 6, 20, 12, 9, 4, 10, 12, 20, 20, 6, 20, 4, 7, 20, 20, 13, 14, 12, 12, 20, 10, 4, 20, 14, 20, 20, 3, 20, 18, 1, 4, 20, 19, 19, 10, 12, 12, 14, 10, 12, 17, 7, 12, 9, 12, 5, 10, 12, 4, 9, 9, 10, 16, 3, 7, 7, 7, 12, 4, 7, 12, 12, 7, 5, 4, 12, 12, 16, 7, 5, 7, 4, 12, 12, 7, 12, 7, 7, 12, 12, 12, 12, 12, 12, 7, 12, 9, 7, 12, 10, 4, 12, 8, 12, 12, 2, 2, 8, 12, 16, 12, 12, 2, 1, 6, 7, 4, 12, 12, 7, 10, 12, 12, 19, 4, 12, 12, 12, 16, 9, 12, 12, 12, 2, 9, 12, 7, 12, 12, 12, 8, 12, 4, 7, 9, 5, 9, 10, 12, 12, 12, 12, 12, 12, 12, 16, 18, 12, 2, 4, 3, 7, 4, 12, 2, 12, 4, 16, 18, 12, 12, 3, 12, 12, 12, 1, 2, 12, 12, 18, 16, 12, 10, 8, 1, 12, 9, 16, 16, 10, 2, 8, 16, 12, 4, 4, 12, 19, 10, 12, 7, 16, 7, 7, 8, 11, 4, 10, 4, 12, 4, 2, 12, 12, 8, 8, 20, 10, 8, 12, 18, 7, 16, 12, 6, 7, 9, 3, 12, 12, 16, 4, 4, 3, 12, 10, 12, 12, 9, 9, 4, 7, 3, 7, 4, 2, 12, 12, 12, 18, 7, 3, 9, 16, 9, 12, 7, 12, 12, 12, 9, 18, 18, 19, 12, 14, 12, 14, 14, 4, 12, 12, 7, 12, 14, 18, 1, 4, 4, 12, 8, 4, 9, 12, 9, 17, 4, 16, 14, 12, 4, 6, 4, 5, 7, 3, 9, 10, 17, 7, 7, 12, 9, 4, 5, 7, 17, 5, 9, 18, 5, 8, 4, 3, 14, 7, 2, 12, 10, 2, 8, 4, 14, 3, 5, 12, 3, 7, 7, 14, 7, 5, 10, 16, 2, 9, 18, 7, 5, 3, 17, 17, 1, 14, 4, 7, 3, 8, 6, 14, 9, 3, 16, 8, 4, 7, 10, 10, 4, 4, 7, 5, 8, 4, 12, 3, 7, 2, 16, 19, 2, 8, 10, 4, 12, 10, 2, 9, 4, 1, 16, 3, 5, 3, 12, 16, 10, 7, 14, 8, 10, 7, 12, 12, 16, 18, 4, 5, 7, 9, 16, 4, 8, 8, 16, 14, 7, 2, 17, 8, 17, 17, 3, 17, 3, 2, 17, 17, 3, 7, 7, 9, 6, 4, 16, 17, 16, 7, 7, 7, 5, 14, 16, 17, 17, 7, 7, 6, 17, 12, 12, 3, 8, 5, 4, 12, 16, 1, 17, 10, 10, 16, 12, 9, 19, 7, 2, 12, 10, 10, 4, 16, 10, 9, 3, 7, 10, 9, 12, 1, 7, 10, 8, 1, 3, 4, 9, 18, 19, 4, 17, 12, 12, 12, 16, 2, 12, 4, 4, 12, 14, 12, 4, 12, 4, 16, 4, 4, 4, 4, 4, 4, 19, 19, 14, 4, 6, 19, 12, 17, 16, 1, 8, 4, 8, 18, 4, 4, 4, 16, 1, 9, 2, 20, 8, 12, 3, 8, 8, 10, 12, 8, 8, 8, 12, 8, 8, 8, 14, 12, 8, 8, 4, 16, 8, 12, 2, 8, 10, 8, 19, 10, 8, 18, 18, 12, 7, 18, 19, 8, 10, 9, 8, 18, 8, 4, 2, 12, 18, 12, 6, 12, 5, 8, 8, 8, 12, 12, 9, 12, 12, 12, 4, 7, 7, 12, 7, 7, 17, 7, 10, 9, 12, 7, 7, 7, 12, 7, 12, 2, 12, 12, 16, 10, 7, 8, 17, 2, 17, 7, 5, 17, 10, 5, 5, 4, 7, 18, 17, 5, 5, 5, 5, 8, 19, 5, 17, 19, 5, 5, 5, 5, 19, 5, 5, 17, 5, 5, 5, 17, 9, 17, 5, 16, 5, 10, 17, 19, 5, 5, 16, 13, 13, 1, 12, 13, 16, 4, 7, 19, 8, 12, 14, 9, 10, 2, 1, 12, 4, 19, 9, 12, 2, 19, 16, 12, 11, 12, 14, 10, 16, 1, 12, 9, 12, 9, 12, 19, 10, 12, 9, 19, 7, 2, 14, 7, 19, 10, 16, 7, 2, 12, 12, 7, 2, 18, 10, 12, 17, 12, 10, 16, 12, 16, 16, 14, 2, 12, 19, 10, 10, 12, 16, 16, 4, 20, 16, 16, 12, 16, 12, 16, 12, 16, 12, 13, 16, 16, 4, 3, 8, 10, 16, 8, 8, 1, 19, 1, 20, 3, 7, 10, 10, 16, 2, 10, 12, 16, 10, 10, 7, 4, 10, 16, 8, 8, 4, 9, 9, 8, 3, 6, 16, 6, 4, 9, 6, 4, 11, 8, 4, 7, 10, 19, 9, 3, 8, 4, 14, 9, 2, 9, 10, 8, 3, 12, 4, 12, 14, 3, 4, 7, 7, 7, 7, 7, 3, 7, 8, 7, 3, 8, 14, 5, 5, 4, 17, 5, 17, 12, 17, 5, 17, 2, 15, 2, 8, 16, 11, 12, 2, 8, 7, 14, 19, 9, 12, 1, 4, 16, 19, 4, 18, 9, 12, 9, 12, 12, 12, 4, 4, 4, 4, 8, 11, 10, 12, 9, 8, 12, 2, 2, 9, 11, 2, 2, 8, 4, 2, 4, 4, 6, 4, 7, 4, 4, 4, 10, 6, 16, 4, 4, 4, 17, 12, 3, 4, 17, 9, 2, 13, 3, 6, 2, 9, 9, 16, 3, 12, 2, 12, 12, 8, 18, 10, 9, 9, 12, 12, 12, 12, 18, 7, 19, 12, 8, 18, 12, 7, 7, 5, 16, 7, 12, 1, 1, 4, 12, 12, 19, 12, 1, 12, 12, 3, 12, 12, 14, 3, 12, 16, 12, 10, 1, 12, 8, 10, 8, 8, 12, 12, 2, 19, 12, 14, 13, 18, 16, 6, 9, 19, 4, 4, 19, 9, 4, 9, 4, 16, 16, 19, 14, 1, 9, 1, 1, 1, 1, 11, 6, 8, 12, 2, 12, 10, 19, 19, 12, 11, 10, 11, 14, 14, 8, 6, 3, 12, 4, 16, 1, 2, 17, 6, 16, 12, 12, 9, 16, 12, 1, 7, 9, 10, 9, 6, 8, 6, 7, 4, 12, 1, 19, 2, 7, 12, 2, 19, 19, 7, 19, 19, 12, 8, 19, 12, 4, 9, 12, 12, 7, 10, 12, 11, 4, 9, 4, 9, 9, 14, 14, 2, 2, 16, 9, 19, 12, 8, 2, 9, 18, 2, 12, 12, 10, 9, 9, 6, 9, 2, 2, 2, 18, 9, 2, 2, 9, 9, 2, 2, 18, 2, 12, 16, 18, 12, 18, 9, 12, 2, 18, 2, 18, 18, 19, 18, 18, 18, 19, 12, 18, 18, 18, 18, 18, 4, 18, 9, 4, 19, 12, 9, 18, 18, 18, 17, 8, 11, 4, 12, 12, 12, 9, 19, 16, 16, 12, 12, 10, 16, 2, 12, 18, 10, 4, 2, 9, 19, 8, 4, 8, 9, 9, 9, 13, 4, 9, 4, 9, 9, 9, 6, 12, 16, 9, 9, 9, 4, 6, 9, 9, 16, 8, 9, 9, 13, 9, 9, 10, 12, 4, 4, 9, 12, 9, 18, 18, 19, 18, 18, 19, 12, 19, 4, 12, 14, 12, 19, 1, 2, 18, 18, 14, 12, 20, 20, 9, 13, 18, 18, 1, 1, 1, 4, 18, 4, 6, 18, 18, 18, 18, 18, 2, 12, 18, 18, 1, 18, 18, 18, 12, 18, 18, 18, 18, 9, 18, 18, 12, 8, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 14, 2, 7, 7, 13, 19, 12, 3, 20, 17, 3, 14, 20, 16, 3, 3, 5, 4, 4, 3, 3, 7, 7, 10, 7, 7, 3, 3, 8, 7, 7, 7, 14, 8, 7, 8, 12, 10, 4, 14, 18, 8, 12, 14, 12, 12, 7, 4, 18, 14, 12, 2, 17, 8, 20, 9, 6, 8, 20, 4, 8, 14, 4, 12, 12, 10, 14, 12, 19, 12, 9, 8, 3, 12, 6, 4, 19, 19, 19, 19, 19, 12, 14, 4, 7, 18, 19, 12, 9, 19, 1, 4, 1, 19, 15, 15, 2, 4, 2, 4, 12, 18, 13, 13, 12, 19, 12, 9, 9, 16, 8, 16, 13, 16, 13, 19, 16, 13, 18, 12, 11, 16, 13, 9, 9, 9, 19, 10, 19, 4, 19, 19, 12, 4, 1, 18, 7, 18, 18, 6, 8, 18, 18, 19, 8, 19, 4, 12, 12, 10, 6, 18, 9, 8, 9, 8, 4, 3, 4, 11, 10, 17, 14, 17, 12, 17, 3, 4, 14, 6, 7, 3, 7, 19, 9, 19, 2, 12, 19, 12, 3, 8, 19, 3, 19, 9, 9, 19, 12, 19, 4, 19, 9, 19, 4, 19, 19, 2, 7, 4, 18, 18, 14, 8, 3, 19, 16, 2, 18, 1, 1, 10, 1, 1, 10, 8, 8, 2, 13, 12, 1, 12, 19, 17, 17, 6, 4, 16, 12, 12, 3, 12, 12, 3, 12, 16, 8, 1, 7, 9, 14, 2, 7, 12, 19, 19, 17, 10, 8, 18, 12, 6, 6, 12, 1, 10, 17, 20, 18, 18, 18, 10, 19, 5, 8, 8, 19, 5, 3, 5, 4, 6, 2, 7, 7, 12, 7, 9, 9, 10, 15, 12, 8, 18, 18, 11, 19, 4, 12, 12, 7, 12, 11, 14, 12, 11, 18, 10, 18, 18, 11, 6, 6, 19, 8, 4, 19, 1, 16, 18, 8, 18, 3, 9, 1, 12, 10, 8, 7, 7, 10, 10, 4, 14, 16, 14, 6, 2, 2, 2, 2, 18, 6, 16, 10, 14, 4, 12, 14, 4, 16, 20, 20, 20, 19, 6, 18, 6, 7, 8, 7, 9, 7, 7, 8, 14, 18, 1, 6, 9, 14, 6, 7, 12, 14, 6, 6, 19, 16, 4, 13, 19, 4, 11, 18, 8, 18, 2, 9, 6, 7, 2, 16, 12, 12, 12, 8, 9, 8, 16, 8, 19, 14, 12, 11, 11, 19, 2, 18, 10, 18, 1, 16, 9, 10, 3, 20, 6, 20, 15, 7, 8, 4, 9, 8, 7, 4, 19, 19, 19, 19, 19, 12, 19, 19, 19, 19, 14, 13, 13, 12, 12, 8, 15, 15, 8, 19, 12, 6, 18, 18, 20, 18, 18, 18, 4, 18, 18, 6, 12, 16, 16, 11, 8, 12, 14, 9, 4, 12, 3, 7, 7, 10, 10, 12, 12, 13, 16, 10, 10, 12, 17, 14, 20, 11, 14, 14, 14, 20, 14, 9, 8, 5, 4, 12, 10, 14, 14, 4, 12, 19, 16, 19, 16, 7, 16, 1, 10, 16, 14, 13, 9, 13, 8, 16, 7, 7, 7, 20, 20, 2, 20, 4, 18, 18, 18, 8, 14, 4, 12, 18, 18, 18, 18, 14, 12, 16, 3, 8, 16, 4, 6, 4, 13, 13, 13, 8, 19, 19, 8, 10, 2, 4, 3, 10, 1, 19, 19, 15, 7, 10, 7, 8, 14, 14, 14, 18, 18, 2, 2, 2, 19, 2, 2, 20, 13, 13, 13, 18, 12, 1, 19, 19, 19, 19, 19, 13, 10, 9, 20, 20, 20, 20, 20, 20, 17, 18, 6, 12, 9, 9, 19, 12, 18, 13, 18, 12, 16, 18, 20, 18, 16, 12, 16, 1, 10, 9, 18, 18, 8, 9, 8, 8, 8, 18, 18, 18, 9, 20, 20, 18, 18, 18, 18, 8, 18, 13, 4, 8, 10, 18, 18, 17, 18, 18, 18, 18, 18]


In [61]:
modsizes=[]
for i in range(0,max(partvals)):
    modsizes.append(partvals.count(i))
print(modsizes)
print('There are',len(modsizes),'modules.')

[8, 114, 208, 237, 487, 233, 313, 306, 285, 262, 268, 149, 541, 37, 123, 10, 326, 92, 194, 151]
There are 20 modules.


In [84]:
from numpy import linalg as LA
w, v = LA.eig(Adj)
w=[l/max(w,key=abs) for l in w]
expdiag=[np.exp(l) for l in w]
mat1=np.matmul(v,np.diag(expdiag))
mat2=np.matmul(mat1,np.linalg.inv(v))



In [109]:
subgraph(Adj,4000,3000)

(3.8219824711766687e-13-4.789921664855036e-31j)


In [14]:
newGnodes=list(G.nodes())
SOD1_index=newGnodes.index('4932.YJR104C')
newGnodes[SOD1_index]
SOD1_module=partvals[247]
print('SOD1 is in module',SOD1_module)
print('Module',SOD1_module,'has',modsizes[SOD1_module],'nodes')

SOD1 is in module 9
Module 9 has 262 nodes
